# Technical Analysis using TaLib

Roboquant comes out of box with support for technical analysis. It does this by having dedicate support for the popular technical analysis libraries TaLib. TaLib contains over 150 technical indicators. These are indicators ranging from RSI and Bollinger Bands to Candlestick pattern recognition. 

Using the TA-Lib library directly can be difficult due to its C++ inspired API. But roboquant makes it easy by providing a convenient wrapper.

In [1]:
%use roboquant(3.0.0-SNAPSHOT)
Welcome()

roboquant 3.0.0-SNAPSHOT 
 build: 2025-05-06T06:39:03Z 
 home: /Users/peter/.roboquant 
 os: Mac OS X 15.4.1 
 jvm: OpenJDK 64-Bit Server VM 17.0.14 
 memory: 3256 MB 
 cpu cores: 8

In [2]:
// We first setup the feed we'll use in all examples below
val feed = AvroFeed.sp25()

We start by defining the  strategy using TALib indicators we want to test in combination with the `TaLibStrategy` class. There are three steps:

1) Create an instance of `TaLibStrategy`.
2) Define what you logic to use for generating BUY signals. This can be a combination of indicators, and you can also include regular Kotlin code. The only restriction is that the last statement has to be a boolean; true if you want to BUY, false otherwise.
3) Define what you logic to use for generating SELL signals. The same rules and conditions apply as for the BUY method.

The indicators in the code below just serve as an example of how to use them. They don't make much sense as a trading strategy.

Also note that although not common practice, the framework doesn't stop you from using bullish indicators to generate a SELL signal and bearish indicators to generate a BUY signal. 

In [3]:
val strategy = TaLibStrategy()

// We want to generate a BUY signal if we detect either one of the following two candlestick pattern: Morningstar or Three WhiteSoldiers
strategy.buy {
    cdlMorningStar(it) || cdl3WhiteSoldiers(it)
}

// We want to generate a SELL signal if we detect the candlestick pattern Two Crows and also observe that the 5 day weighted average is below the 9 day weighted average 
strategy.sell {
    cdl2Crows(it) && ema(it.close, 5) < ema(it.close, 9)
}



Besides strategies, we can also use TALib indicators as metrics that are captured during a run. 

The following code creates a custom metric that calculates the Money Flow Index (aka mfi), one of the many indicators that comes with TALib. 

This metric will be automatically applied to each asset when used in a run.

Now we can use this strategy and metric just like any other strategy and metric. 

In this case, we run a back-test against the S&P500 feed. 


In [6]:
run(feed, strategy)

last update  : 2024-12-31T21:00:00Z
cash         : USD 774,233.09
buying Power : USD 774,233.09
equity       : USD 1,242,612.34
positions    : 49@V, 228@AVGO, 691@NVDA, 82@AAPL, 75@PG, 36@BRK.B, 73@GOOGL, 28@MA, 63@JNJ, 73@GOOG, 89@ABBV, 18@COST, 59@AMZN, 224@WMT, 44@META, 34@MSFT, 29@NFLX, 79@JPM, 124@MRK, 20@UNH, 35@HD, 65@CVX, 97@XOM
open orders  : 

Now lets see what the outcome is. Spoiler alert, this strategy won't make you rich ;)

Let find out which trade generated most profit (realized PnL) and plot mfi for that asset

## Bollinger Bands
This example is more realistic and shows how to implement a strategy based on Bollinger Bands. The underlying assumptions are:

1. If the current price is higher than the upper bound, it will continue to go up.
2. If the current price is lower than lower bound, it will continue to go down.

Of course you can easily reverse this logic from trend following to mean reversion if you prefer and compare the outcomes.

In [7]:
// Define a strategy with maximum of bars of history. 
val barCount = 20
val strategy = TaLibStrategy(barCount)

// We want to BUY if current price is higher than the Bollinger Bands higher bound 
strategy.buy {
    val (h, m, l) = bbands(it, barCount)
    it.close.last() > h 
}

// We want to SELL if current price is lower than the Bollinger Bands lower bound 
strategy.sell {
    val (h, m, l) = bbands(it, barCount)
    it.close.last() < l   
}

In [12]:
val journal = MemoryJournal(AccountMetric())
run(feed, strategy, journal = journal)

last update  : 2024-12-31T21:00:00Z
cash         : USD 913,442.79
buying Power : USD 913,442.79
equity       : USD 1,118,585.98
positions    : 11@COST, 14@NFLX, 0@BRK.B, 23@MA, 160@WMT, 39@V, 21@UNH, 28@HD, 0@PG, 0@JNJ, 57@GOOG, 58@GOOGL, 85@NVDA, 98@XOM, 0@CVX, 47@AAPL, 23@TSLA, 0@MRK, 59@AMZN, 26@MSFT, 0@LLY, 54@ABBV, 17@META, 60@AVGO, 0@JPM
open orders  : -11@COST, -26@MSFT

In [13]:
val equity = journal.getMetric("account.equity")
TimeSeriesChart(equity)

1005

<!-- roboquant TimeSeriesChart -->

# Indicators

In [14]:
// Indicator that returns multiple values
val asset = Stock("GOOG")
val data = feed.apply(TaLibIndicator.bbands(), asset)
TimeSeriesChart(data)

<!-- roboquant TimeSeriesChart -->

In [15]:
// Compare indicators across different assets
val rsi = TaLibIndicator.rsi(20)
val apple = Stock("AAPL")
val tesla = Stock("TSLA")
val data = feed.apply(rsi, apple, tesla)
TimeSeriesChart(data)

<!-- roboquant TimeSeriesChart -->

In [16]:
// see where sma crossovers appear
val data1 = feed.apply(TaLibIndicator.sma(15), apple)
val data2 = feed.apply(TaLibIndicator.sma(50), apple)
TimeSeriesChart(data1 + data2)

<!-- roboquant TimeSeriesChart -->

In [17]:
// Plot an indicator on a PriceBarChart
val indicators = arrayOf(TaLibIndicator.bbands(20))
PriceBarChart(feed, tesla, indicators = indicators)

<!-- roboquant PriceBarChart -->